# Armbot disinfection

In [1]:
import pickle
import numpy as np
import visibility
from importlib import reload
reload(visibility)
from visibility.visibility import Visibility
from klampt import vis
import os
from scipy.spatial.transform import Rotation as R
import random
import numpy as np
import math
import time
import networkx as nx
from matplotlib import cm
import klampt
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.model.create import *
from klampt.robotsim import Geometry3D,WorldModel
from klampt.vis import colorize
from klampt.model import trajectory
from threading import Thread,Lock,Event
import pathlib
from experiments2d.getToursAndPaths import getTour, readTourFile, getPathFromPrm, getFinalPath

from IPython import embed

enabling speedups
***  klampt.vis: using Qt5 as the visualization backend  ***


In [2]:
configs_file = 'configs.p'
robot_file = "./data/armbot.rob"

results_dir = './3D_results/Semantic/scene0015_00/surface_agnostic_1_minutes/armbot'
mesh_file = '/data/scannet_subset_all_gt/gt_scene0015_00.ply'

def generate_video(mesh_file = mesh_file,
                   robot_file = "./data/armbot.rob",
                   movie_dir = './media/movies/armbot/surface_agnostic_1_minutes',
                   tour_file = 'armbot_currTSP.txt',
                   results_dir = results_dir,
                   resolution = 330,
                   skip = 1,
                   dt = 0.4,
                   max_step = 20,
                   scale = 4,
                   linknum = 11):

    pcd_file = results_dir + "/armbot_used_points_{}_divs.pcd".format(resolution)
    reachable_points_file = results_dir + '/armbot_reachable_{}_divs.p'.format(resolution)
    irradiance_file = results_dir + '/armbot_irradiance_matrix_{}_divs.p'.format(resolution)
    sampling_places_file = results_dir + '/armbot_sampling_places_{}_divs.p'.format(resolution)
    configs_file = results_dir + '/armbot_configs_{}_divs.p'.format(resolution)
    solutions_file = results_dir + '/armbot_solutions_{}_divs.p'.format(resolution)
    adjacency_file = results_dir + '/armbot_adjacency_{}_divs.p'.format(resolution)
    roadmap_file = results_dir + '/armbot_roadmap_{}_divs.p'.format(resolution)
    node_coords_file = results_dir + '/armbot_node_coords_{}_divs.p'.format(resolution)

    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    reachable = pickle.load(open(reachable_points_file,'rb'))
    configs = pickle.load(open(configs_file,'rb'))
    sampling_places = pickle.load(open(sampling_places_file,'rb'))

    #creating movie directory
    os.makedirs(movie_dir, exist_ok=True)
    # fixing movie paths
    movie_folder = movie_dir
    movie_dir += '/{:06d}.png'
    
    sampling_places = sampling_places[reachable]
    configs = np.array(configs)[reachable].tolist()
    mu_single = 60*60*80*0.5*irradiance_matrix
    print("\n\n number of reachable points : {} \n".format(np.sum(reachable)))

    #process irradiation spots
    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))

    vis_tester =Visibility(mesh_file,res = 512, useShader = True,createWnd = True)
    m = vis_tester.mesh
    
    face_ht_prob = m.vanilla_high_touch_prob()
    lt_idx = (face_ht_prob == 0)
    ht_idx = (face_ht_prob != 0)
    face_ht_prob[lt_idx] = 1
    face_ht_prob[ht_idx] = 0
    areas = m.area()
    
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_areas = areas[visible_points]

    total_visible_areas = visible_areas.sum()
    total_visible_areas

    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()

    visible_irradiance = irradiance_matrix[:,visible_points]


    visible_areas = areas[visible_points]
    # we then define the visible area weights:

    visible_area_weights = np.exp(100*(visible_areas/visible_areas.sum()))
    visible_area_weights[:] = 1


    solutions = pickle.load(open(solutions_file,'rb'))
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_irradiance = irradiance_matrix[:,visible_points]

    # we now calculate the euclidean distance matrices for all the non-zero points:
    times = solutions[0]
    total_time = times.sum()
    points_mask = times>0.5
    used_points = sampling_places[points_mask,:]
    print(used_points.shape)
    mus = 80*np.array(np.matmul(visible_irradiance.todense().transpose(),times)).flatten()
    coverage = visible_areas[mus > 280].sum()/243.6
    print('Total Environment Coverage = {:.2f}%'.format(100*coverage))

    adjacency_matrix = pickle.load(open(adjacency_file,'rb'))
    roadmap = pickle.load(open(roadmap_file,'rb'))
    node_coords = pickle.load(open(node_coords_file,'rb'))


    distances = np.zeros(shape = (adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
    distances[1:,1:] = 100*adjacency_matrix.copy()

    tour = readTourFile(tour_file, used_points)
    tour = (np.array(tour[1:])-1).tolist()
    indices = np.array(range(sampling_places.shape[0]))
    used_indices = indices[points_mask]
    tour_indices = used_indices[tour]

    # We then calculate the total distance travelled - and compute final trajectory:
    total_distance = 0 
    final_trajectory = []
    for i in range(len(tour)-1):
        current_point = tour[i]
        next_point = tour[i+1]
        total_distance += adjacency_matrix[current_point,next_point]
        nodes_path = nx.algorithms.shortest_path(roadmap,source = current_point,target = next_point,weight = 'weight')
        traj = [node_coords[j][:12] for j in nodes_path]
        final_trajectory.append(traj[:-1])
    print("Total Distance Travelled by the EE = {} | Total Time Spent in Transit = {} ".format(total_distance,total_distance/30))

    #prepping animations
    time_allowances = np.linspace(0.00,0.5,50)
    times = solutions[0]
    partial_times  = []
    for allowance in time_allowances:
        partial_time = np.zeros(sampling_places.shape[0])

        for k in range(len(tour_indices)-1):
            total_time = times[tour_indices[:k+1]].sum()/(3600)
            if(total_time >= allowance):
                k = k-1
                break
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        print(partial_time.sum()/3600)
        partial_times.append(partial_time)

    # trajectory_times:

    total_times = []
    for k in range(len(tour_indices)):
        partial_time = np.zeros(sampling_places.shape[0])
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        total_times.append(partial_time.sum()/3600)
    #     print(total_times[-1])
        partial_times.append(partial_time)
    total_times = np.array(total_times)

    # we then determine the strategic stops
    first = np.argmin(np.abs(total_times-0.125))
    second = np.argmin(np.abs(total_times-0.25))
    third =  np.argmin(np.abs(total_times-0.375))
    fourth = total_times.shape[0] - 1



    stop_spots = [first,second,third,fourth]
    timed_trajectory = []
    trajectory_times = []
    prev_time = 0
    for index,path in enumerate(final_trajectory):
        section_length = len(path)
        section_times = np.zeros(section_length) + prev_time
        section_times = section_times + np.arange(section_length)*((total_times[index]-prev_time)/section_length)
        section_times = section_times*3600
        prev_time = total_times[index]
        timed_trajectory.extend(path)
        trajectory_times.extend(section_times)

    # setting up basic animations


    vis.init('PyQt')
    # del robot
    a = Geometry3D()
    res = a.loadFile(mesh_file)

    b = Geometry3D()
    res = b.loadFile(pcd_file)
    if(not res):
        print('failed loading geometry!')
    w = WorldModel()

    res = w.loadElement(robot_file)
    print(res)
    robot = w.robot(0)
    cfig = robot.getConfig()
    cfig[2] = 0.08
    robot.setConfig(cfig)
    w.makeRigidObject("a")
    w.rigidObject(0).geometry().set(a)
    w.makeRigidObject("b")
    w.rigidObject(1).geometry().set(b)

    robotTraj = RobotTrajectory(robot,milestones = timed_trajectory,times = trajectory_times).discretize(dt)
    linkTraj = robotTraj.getLinkTrajectory(linknum,dt) #.getPositionTrajectory([0,0,0])

    times = solutions[0]
    final_times = []
    for this_time in robotTraj.times:
        for j in range(len(tour_indices)):
            partial_time = np.zeros(sampling_places.shape[0])
            partial_time[tour_indices[:j+1]] = times[tour_indices[:j+1]]
            total_time = partial_time.sum()
            if(total_time >= this_time):
                break
        partial_time[tour_indices[j]] = this_time-times[tour_indices[:j]].sum()
        final_times.append(partial_time)
    final_times = np.array(final_times)
    # robot,w
    value = None
    cmap = None
    feature = None
    vis.add("A",a)
    # vis.add("B",b,pointSize = 15)
    vis.add("robot",robot)
    curr_milestones = sampling_places[tour_indices[:1],:]
    full_trajectory = []
    for i in final_trajectory:
        full_trajectory.extend(i)
    traj = trajectory.Trajectory(milestones= full_trajectory)#curr_milestones.tolist())
    
    # we then calculate the percentage of disinfection fluence:
    # disinfected_vecs = []
    partial_times_array = np.array(partial_times).transpose()
    # for times in partial_times:
    # mu = 80*np.matmul(visible_irradiance.transpose().toarray(),final_ties)
    # disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()

    # we then add all zeros to begin the illustration
    # tmp = [np.zeros(disinfected_vecs[0].shape)]
    # tmp[0][0] = 1
    # disinfected_vecs_new = disinfected_vecs.copy()
    vis.visualization.setBackgroundColor(1, 1, 1, a=1)

    vis.show()
    robot.link(11).appearance().setColor(210/255,128/255,240/255,1)

    # eliminating draw distance
    vis.lock()
    vp = vis.getViewport()
    vp.clippingplanes = [0.1,10000]
    tform = pickle.load(open('tform.p','rb'))
    vp.setTransform(tform)
    vp.w = 1853//scale
    vp.h = 1123//scale
    # vis.setViewport(vp)
    vis.scene().setViewport(vp)
    vis.unlock()
    time.sleep(1)
    new_partial_times = final_times
    max_t = 0
    for i in partial_times:
        max_t = max(max_t,np.max(i))

        
        #animation steps
    tholds = [0.25,0.5,0.75,0.999995]
    thold_index = 0
    thold = tholds[thold_index]
    # i = 56
    # while vis.shown():
    done = get_done_set(movie_folder)
    total_frames = set(range(len(robotTraj.milestones)))
    print(len(total_frames),len(done))
    to_do = total_frames - done
    print('todo = : {}'.format(to_do))
    sc = Screenshotter(movie_dir)
    time.sleep(5)
    for i in sorted(list(to_do))[skip::max_step]:
        times = new_partial_times[i]
        disinfection_fractions = areas.copy()
        disinfection_fractions[:] = 0
        mu = 80*np.matmul(visible_irradiance.transpose().toarray(),times.transpose())
        disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()
        if(i == 0):
            disinfected_vecs[0] = 100

        disinfection_fractions[visible_points] = disinfected_vecs
        
        # Manually set LT surface to LT
        disinfection_fractions[lt_idx] = 1

        start = time.time()

        # Translate value to HT colors
        # 1 is green
        # 0 is red
        cm_interpolator = cm.get_cmap('RdYlGn')
        face_colors = np.asarray(disinfection_fractions)
        face_colors = cm_interpolator(face_colors)

        face_colors[lt_idx, 3] = 0
        a_app = colorize.colorize(w.rigidObject(0),face_colors,feature='faces',lighting=[0,0,-1])
        
        end = time.time()
        vis.lock()
        traj = Trajectory(milestones = linkTraj.getPositionTrajectory([0,0,0]).milestones[:i])

        vis.add("A",a_app)
        
        if(i!= 0 ):
            vis.add("traj",traj)
        vis.addText('disinfection_time','Time = {:.2f} minutes'.format(times.sum()/60),position = [10//scale,50//scale],size = 36//scale)
        robot.setConfig(robotTraj.milestones[i])

        vis.unlock()
        sc.event.clear()
        sc.i = i
        vis.threadCall(sc.take_screenshot)
        sc.event.wait()
        print('\n {} took {}\n '.format(i,end-start))
    vis.kill()
    
class Screenshotter:
    def __init__(self,movie_dir):
        self.event = Event()
        self.movie_dir = movie_dir
        self.i = 0
        self.event.clear()
    def take_screenshot(self):
        vis.scene().program.save_screen(self.movie_dir.format(self.i))
        self.event.set()
        
def get_done_set(movie_dir):
    from glob import glob
    import pandas as pd
    a = glob(movie_dir + '/*.png')
    if(a):
        a = pd.Series(a)
        b = a.str.split('/',expand = True).iloc[:,-1]
        c = b.str.split('.', expand = True).iloc[:,0].astype(int)
        d = set(c)
        return d
    else:
        return set({})

In [ ]:
generate_video(skip = 1,max_step = 1,dt = 0.5,scale = 1)



 number of reachable points : 259 

(34, 3)
Total Environment Coverage = 16.66%
Total Distance Travelled by the EE = 45.001021206358736 | Total Time Spent in Transit = 1.5000340402119579 
0.0
0.010135516545691196
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035103942701654
0.015035

In [4]:
help(vis.init)

Help on function init in module klampt.vis.visualization:

init(backends=None)
    Initializes the vis module using some visualization backend.  `backends`
    can be None, in which case it tries using PyQt, then GLUT, then IPython in
    that order.  It can also be a string or list of strings from the following
    set:
    
    - 'PyQt': uses PyQT + OpenGL
    - 'PyQt4' / 'PyQt5': uses a specific version of PyQT
    - 'GLUT': uses GLUT + OpenGL
    - 'IPython': uses an IPython widget
    - 'HTML': outputs an HTML / Javascript widget



In [4]:
help(colorize.colorize)

Help on function colorize in module klampt.vis.colorize:

colorize(object, value, colormap=None, feature=None, vrange=None, lighting=None)
    Colorizes an object according to some value.  Useful for making
    heatmaps, false color images, etc.  Can only be used with point clouds
    and triangle meshes.
    
    Examples:
    
    - ``colorize(point_cloud,'z','plasma')``: sets the point cloud to a rainbow
      colorization (the 'plasma' colormap in Matplotlib) depending on z.
    
    - ``colorize(trimesh,'nz')``: colorizes the trimesh according to the z
      component of each triangle normal.
    
    - ``colorize(trimesh,[v1_val,v2_val,...,vm_val])``: colorizes each vertex
      of the triangle mesh by the default colormap. (here m is the # of
      vertices)
    
    - ``colorize(trimesh,[t1_rgb,t2_rgb,...,tn_rgb])``: colorizes each triangle
      of the triangle mesh to an assigned color. (here n is the # of
      triangles)
    
    - ``colorize(trimesh,'index','random',featur

In [ ]:
help(vis.threadCall)